# Preload precipitation files

Since loading from the original COSMO GRIB files takes quite a long time, I will try to preload them in a separate file. This should hopefully speed up my analysis.

In [20]:
# Imports
import helpers; reload(helpers)
from helpers import *
from config import *
import os
import numpy as np
import datetime as dt
from cosmo_utils.helpers import ddhhmmss, make_timelist, yyyymmddhhmmss_strtotime, yyyymmddhhmmss
import timeit

In [3]:
exp_ids = ['DA_REF', 'DA_PSPv2', 'DA_REF_TL500', 'DA_PSPv2_TL500', 'noDA_PSPv2']
date_start = '20160526000000'
date_stop = '20160608120000'
hint = 12
fclt = 24   # Forecast lead time

In [4]:
date_list = make_timelist(
    yyyymmddhhmmss_strtotime(date_start),
    yyyymmddhhmmss_strtotime(date_stop),
    dt.timedelta(hours=hint),
)

In [5]:
date_list[:5]

[datetime.datetime(2016, 5, 26, 0, 0),
 datetime.datetime(2016, 5, 26, 12, 0),
 datetime.datetime(2016, 5, 27, 0, 0),
 datetime.datetime(2016, 5, 27, 12, 0),
 datetime.datetime(2016, 5, 28, 0, 0)]

## Preload deterministic forecasts

In [5]:
def preload_det_prec_for_one_day(exp_id, date_str, fclt=24):
    data_dir = datadir + exp_id + '/'
    for h in range(1, fclt +1):
        hstr = ddhhmmss(dt.timedelta(hours=h))
        try:
            p = load_det(data_dir, date_str, hstr, return_array=True)
        except Exception:
            print 'File missing:', data_dir, date_str, hstr
            p = np.nan
        if h == 1:  # Allocate output array
            prec = np.ones((fclt, p.shape[0], p.shape[1])) * np.nan
        prec[h - 1] = p
    # Save
    save_fn = savedir_base + exp_id + '/prec_fields/' 
    if not os.path.exists(save_fn):
        os.makedirs(save_fn)
    save_fn += 'det_' +  date_str + '.npy'
    np.save(save_fn, prec)

In [41]:
preload_det_prec_for_one_day('DA_REF', date_start)

initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526

## Preload ensemble forecasts

In [34]:
def preload_ens_prec_for_one_day(exp_id, date_str, fclt=24):
    data_dir = datadir + exp_id + '_ens/'
    for h in range(1, fclt +1):
        hstr = ddhhmmss(dt.timedelta(hours=h))
        try:
            p = np.array(load_ens(data_dir, date_str, hstr, return_array=True))
        except Exception:
            print 'File missing:', data_dir, date_str, hstr
            p = np.nan
        if h == 1:  # Allocate output array
            prec = np.ones((fclt, p.shape[0], p.shape[1], p.shape[2])) * np.nan
        prec[h - 1] = p
    # Save
    save_fn = savedir_base + exp_id + '/prec_fields/' 
    if not os.path.exists(save_fn):
        os.makedirs(save_fn)
    save_fn += 'ens_' +  date_str + '.npy'
    np.save(save_fn, prec)

In [47]:
preload_ens_prec_for_one_day('DA_REF', date_start)

read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:03.646483
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.738707
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:06.395695
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.334186
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.423728
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.442428
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.723308
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.542957
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
re

## Loop over experiments and dates

In [48]:
for exp_id in exp_ids:
    print exp_id
    for date in date_list:
        print date
        date_str = yyyymmddhhmmss(date)
        preload_det_prec_for_one_day(exp_id, date_str)
        preload_ens_prec_for_one_day(exp_id, date_str)

DA_REF
2016-05-26 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/da

reading took 0:00:03.567355
2016-05-26 12:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160526120000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.R

reading took 0:00:04.030199
2016-05-27 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.R

reading took 0:00:03.875127
2016-05-27 12:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160527120000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.R

reading took 0:00:04.095630
2016-05-28 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_REF/20160528000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.R

...

initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00050000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00040000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00060000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608000000/det/lfff00050000_15
initial path:  /project/

initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00040000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00030000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00050000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/DA_PSPv2_TL500/20160608120000/det/lfff00040000_15
initial path:  /project/

reading took 0:00:04.047074
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.162770
noDA_PSPv2
2016-05-26 00:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00030000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00020000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526000000/det/lfff00040000_15


reading took 0:00:04.344571
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.216893
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.593591
read to tmp-file /tmp/user/13098/tmp9okG27fieldobj_list_array_met-ws-720e07_10156.npy
reading took 0:00:04.153214
2016-05-26 12:00:00
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526120000/det/lfff00010000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526120000/det/lfff00000000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526120000/det/lfff00020000_15
1h earlier:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526120000/det/lfff00010000_15
initial path:  /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2/20160526120000/det/lfff00030000_15
1h earlier:  /proje

Exception: Requested member file does not exist:/project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data_forecast/noDA_PSPv2_ens//20160526120000/ens016/lfff00120000_15

Some data is missing for the noDA runs. I built in an exception for these cases in the loading functions where missing files are filled with NaNs.

In [ ]:
for date in date_list:
    print date
    date_str = yyyymmddhhmmss(date)
    # preload_det_prec_for_one_day('noDA_PSPv2', date_str)
    preload_ens_prec_for_one_day('noDA_PSPv2', date_str)

2016-05-26 00:00:00
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:01:13.911158
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:01:02.236207
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:01:03.271657
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:59.557993
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:58.649855
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:01:04.582895
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:01:04.632550
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:01:10.955256
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws

## Preload radar observations

In [15]:
def preload_radar_prec_for_one_day(date_str, fclt=24):
    for h in range(1, fclt +1):
        hstr = ddhhmmss(dt.timedelta(hours=h))
        p = load_radar(date_str, hstr, return_array=True);
        if h == 1:  # Allocate output array
            prec = np.empty((fclt, p.shape[0], p.shape[1]))
        prec[h - 1] = p
    # Save
    save_fn = savedir_base + 'radar/prec_fields/' 
    if not os.path.exists(save_fn):
        os.makedirs(save_fn)
    save_fn += 'radar_' +  date_str + '.npy'
    np.save(save_fn, prec)

In [16]:
preload_radar_prec_for_one_day(date_start)

In [17]:
for date in date_list:
    print date
    date_str = yyyymmddhhmmss(date)
    preload_radar_prec_for_one_day(date_str)

2016-05-26 00:00:00
2016-05-26 12:00:00
2016-05-27 00:00:00
2016-05-27 12:00:00
2016-05-28 00:00:00
2016-05-28 12:00:00
2016-05-29 00:00:00
2016-05-29 12:00:00
2016-05-30 00:00:00
2016-05-30 12:00:00
2016-05-31 00:00:00
2016-05-31 12:00:00
2016-06-01 00:00:00
2016-06-01 12:00:00
2016-06-02 00:00:00
2016-06-02 12:00:00
2016-06-03 00:00:00
2016-06-03 12:00:00
2016-06-04 00:00:00
2016-06-04 12:00:00
2016-06-05 00:00:00
2016-06-05 12:00:00
2016-06-06 00:00:00
2016-06-06 12:00:00
2016-06-07 00:00:00
2016-06-07 12:00:00
2016-06-08 00:00:00
2016-06-08 12:00:00


## Preload CAPE and CIN

In [19]:
def preload_det_cape_cin_for_one_day(exp_id, date_str, fclt=24):
    data_dir = datadir + exp_id + '/'
    for h in range(1, fclt +1):
        hstr = ddhhmmss(dt.timedelta(hours=h))
        ca, ci = load_det_cape_cin(data_dir, date_str, hstr, return_array=True)
        ci[ci < 0] = np.nan
        if h == 1:  # Allocate output array
            cape = np.ones((fclt, ca.shape[0], ca.shape[1])) * np.nan
            cin = np.ones((fclt, ca.shape[0], ca.shape[1])) * np.nan
        cape[h - 1] = ca
        cin[h - 1] = ci
    # Save
    save_fn_cape = savedir_base + exp_id + '/cape_fields/' 
    if not os.path.exists(save_fn_cape):
        os.makedirs(save_fn_cape)
    save_fn_cape += 'det_' +  date_str + '.npy'
    np.save(save_fn_cape, cape)
    save_fn_cin = savedir_base + exp_id + '/cin_fields/' 
    if not os.path.exists(save_fn_cin):
        os.makedirs(save_fn_cin)
    save_fn_cin += 'det_' +  date_str + '.npy'
    np.save(save_fn_cin, cin)

In [20]:
for exp_id in exp_ids:
    print exp_id
    for date in date_list:
        print date
        date_str = yyyymmddhhmmss(date)
        preload_det_cape_cin_for_one_day(exp_id, date_str)

DA_REF
2016-05-26 00:00:00
2016-05-26 12:00:00
2016-05-27 00:00:00
2016-05-27 12:00:00
2016-05-28 00:00:00
2016-05-28 12:00:00
2016-05-29 00:00:00
2016-05-29 12:00:00
2016-05-30 00:00:00
2016-05-30 12:00:00
2016-05-31 00:00:00
2016-05-31 12:00:00
2016-06-01 00:00:00
2016-06-01 12:00:00
2016-06-02 00:00:00
2016-06-02 12:00:00
2016-06-03 00:00:00
2016-06-03 12:00:00
2016-06-04 00:00:00
2016-06-04 12:00:00
2016-06-05 00:00:00
2016-06-05 12:00:00
2016-06-06 00:00:00
2016-06-06 12:00:00
2016-06-07 00:00:00
2016-06-07 12:00:00
2016-06-08 00:00:00
2016-06-08 12:00:00
DA_PSPv2
2016-05-26 00:00:00
2016-05-26 12:00:00
2016-05-27 00:00:00
2016-05-27 12:00:00
2016-05-28 00:00:00
2016-05-28 12:00:00
2016-05-29 00:00:00
2016-05-29 12:00:00
2016-05-30 00:00:00
2016-05-30 12:00:00
2016-05-31 00:00:00
2016-05-31 12:00:00
2016-06-01 00:00:00
2016-06-01 12:00:00
2016-06-02 00:00:00
2016-06-02 12:00:00
2016-06-03 00:00:00
2016-06-03 12:00:00
2016-06-04 00:00:00
2016-06-04 12:00:00
2016-06-05 00:00:00
2016

## Preload deterministic first guess

How are the filest structured:
- Directories are 3 hourly (eg 20160526090000)
- They contain lff_prec.* files from -1 to +1h, which contain the accumulation from 0 to +1h
- but laf files from 0 to +2h

So which is the corresponding radar observation?
- The radar observations for a given hour contain the accumulation from -1 to 0h

So the corresponding radar observation is +1h from the first guess forecast. 

For daily saves the first time step is 0 to 1UTC, which would be saved in the 0UTC file. The last time step is 23 to 24UTC which would be in the 23UTC file


In [28]:
d_shape = (461, 421)

In [29]:
def preload_fg_det_prec_for_one_day(exp_id, date_obj, fclt=24):
    # Get 3hrly storage dir
    date_str = date_str = yyyymmddhhmmss(date_obj)
    for h in range(fclt):
        td = dt.timedelta(hours=h)
        t = date_obj + td
        date_ana = yyyymmddhhmmss(t)
        date_fg = yyyymmddhhmmss(t - timedelta(hours=1))
        date_store = yyyymmddhhmmss(t - timedelta(hours=t.hour % 3))
        
        ana_dir = (datadir_da + exp_id + '/' + date_store + '/')
        hstr = ddhhmmss(dt.timedelta(hours=h))
        try:
            p = load_det_da(ana_dir, date_fg, return_array=True)
        except Exception:
            print 'File missing:', ana_dir, date_fg
            p = np.nan
        if h == 0:  # Allocate output array
            prec = np.ones((fclt, d_shape[0], d_shape[1])) * np.nan
        prec[h - 1] = p
    # Save
    save_fn = savedir_base + exp_id + '/prec_fields/' 
    if not os.path.exists(save_fn):
        os.makedirs(save_fn)
    save_fn += 'det_da_' +  date_str + '.npy'
    np.save(save_fn, prec)

## Preload ensemble first guess

In [30]:
n_ens_da = 40

In [31]:
def preload_fg_ens_prec_for_one_day(exp_id, date_obj, fclt=24):
    # Get 3hrly storage dir
    date_str = date_str = yyyymmddhhmmss(date_obj)
    for h in range(fclt):
        td = dt.timedelta(hours=h)
        t = date_obj + td
        date_ana = yyyymmddhhmmss(t)
        date_fg = yyyymmddhhmmss(t - timedelta(hours=1))
        date_store = yyyymmddhhmmss(t - timedelta(hours=t.hour % 3))
        
        ana_dir = (datadir_da + exp_id + '/' + date_store + '/')
        hstr = ddhhmmss(dt.timedelta(hours=h))
        try:
            p = np.array(load_ens_da(ana_dir, date_fg, return_array=True))
        except Exception:
            print 'File missing:', ana_dir, date_fg
            p = np.nan
        if h == 0:  # Allocate output array
            prec = np.ones((fclt, n_ens_da, d_shape[0], d_shape[1])) * np.nan
        prec[h - 1] = p
    # Save
    save_fn = savedir_base + exp_id + '/prec_fields/' 
    if not os.path.exists(save_fn):
        os.makedirs(save_fn)
    save_fn += 'ens_da_' +  date_str + '.npy'
    np.save(save_fn, prec)

In [32]:
for exp_id in exp_ids:
    print exp_id
    for date in date_list[1:]:
        print date
        preload_fg_det_prec_for_one_day(exp_id, date)
        preload_fg_ens_prec_for_one_day(exp_id, date)

DA_REF
2016-05-26 12:00:00
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.942684
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.436049
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.852548
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.726480
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.101300
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.838720
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.509874
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.803486
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array

reading took 0:00:04.857313
2016-05-28 00:00:00
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.086587
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.167996
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.364404
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:03.973538
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:06.030715
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.432071
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.627276
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.280690
read to tmp-file /tmp/user/13098/tmpYFoK

reading took 0:00:05.098363
2016-05-29 12:00:00
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.047234
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:03.969396
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:03.823507
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:03.751779
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:03.808608
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:03.917339
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:03.769957
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:03.804996
read to tmp-file /tmp/user/13098/tmpYFoK

reading took 0:00:05.721551
2016-05-31 00:00:00
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.162955
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.698813
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.722101
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.369356
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:04.586666
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.336036
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.915755
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.308921
read to tmp-file /tmp/user/13098/tmpYFoK

reading took 0:00:05.101481
2016-06-01 12:00:00
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/DA_REF/20160602000000/ 20160601230000
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:07.253857
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:07.317980
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:06.306384
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.457182
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.600675
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.431097
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_list_array_met-ws-720e07_28141.npy
reading took 0:00:05.113870
read to tmp-file /tmp/user/13098/tmpYFoKZKfieldobj_li

...

File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160604120000/ 20160604110000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160604120000/ 20160604120000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160604120000/ 20160604130000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160604150000/ 20160604140000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160604150000/ 20160604150000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160604150000/ 20160604160000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160604180000/ 20160604170000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160604180000/ 20160604180000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160604180000/ 20160604190000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_da

File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160605120000/ 20160605110000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160605120000/ 20160605120000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160605120000/ 20160605130000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160605150000/ 20160605140000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160605150000/ 20160605150000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160605150000/ 20160605160000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160605180000/ 20160605170000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160605180000/ 20160605180000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160605180000/ 20160605190000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_da

File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160606120000/ 20160606110000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160606120000/ 20160606120000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160606120000/ 20160606130000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160606150000/ 20160606140000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160606150000/ 20160606150000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160606150000/ 20160606160000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160606180000/ 20160606170000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160606180000/ 20160606180000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160606180000/ 20160606190000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_da

File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160607120000/ 20160607110000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160607120000/ 20160607120000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160607120000/ 20160607130000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160607150000/ 20160607140000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160607150000/ 20160607150000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160607150000/ 20160607160000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160607180000/ 20160607170000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160607180000/ 20160607180000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160607180000/ 20160607190000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_da

File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160608120000/ 20160608110000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160608120000/ 20160608120000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160608120000/ 20160608130000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160608150000/ 20160608140000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160608150000/ 20160608150000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160608150000/ 20160608160000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160608180000/ 20160608170000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160608180000/ 20160608180000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_data/data/noDA_PSPv2/20160608180000/ 20160608190000
File missing: /project/meteo/w2w/A6/S.Rasp/kenda_psp_da